### Load in all the data 

In [25]:
import pandas as pd
import numpy as np
from scipy.stats import shapiro
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from rdkit import Chem
from rdkit.Chem import Descriptors, AllChem
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, classification_report, f1_score, recall_score, precision_score
from imblearn.over_sampling import SMOTE, RandomOverSampler
from xgboost import XGBClassifier
import warnings
from itertools import product
from mordred import Calculator, descriptors
import joblib


#For reproducibility
np.random.seed(42)

#Load the data
file_path = r"tested_molecules.csv"
data = pd.read_csv(file_path)
test_path = r"untested_molecules-3.csv"
untested_data = pd.read_csv(test_path)
untested_data = untested_data.drop(columns=['PKM2_inhibition', 'ERK2_inhibition'])
data_all = data
data = data.drop(columns=['PKM2_inhibition', 'ERK2_inhibition'])
data 

,SMILES
0,C=C(C)c1nc(N)nc(N)n1
1,C=C(Cl)COc1ccc2c(C)cc(=O)oc2c1
2,C=CCNC(=O)CCCC(=O)NCC=C
3,C=CCOn1c(=O)c(C)[n+]([O-])c2ccccc21
4,C=CCn1cc(Cl)c(=O)n(CC=C)c1=O
...,...
1111,O=C1c2ccccc2[C@H](Nc2ccc3c(c2)OCCO3)N1Cc1ccco1
1112,O=S(=O)(Nc1cccc(-c2cn3ccsc3[nH+]2)c1)c1ccc(F)cc1
1113,Oc1c(C[NH+]2CCN(c3ccccn3)CC2)cc(Cl)c2cccnc12
1114,c1ccc(-c2csc(N3CCN(c4ccccn4)CC3)n2)cc1


In [26]:
data_tested_untested = pd.concat([data, untested_data], axis=0)
data_tested_untested 

,SMILES
0,C=C(C)c1nc(N)nc(N)n1
1,C=C(Cl)COc1ccc2c(C)cc(=O)oc2c1
2,C=CCNC(=O)CCCC(=O)NCC=C
3,C=CCOn1c(=O)c(C)[n+]([O-])c2ccccc21
4,C=CCn1cc(Cl)c(=O)n(CC=C)c1=O
...,...
4455,c1cncc([C@H]2CCCC[NH2+]2)c1
4456,COc1cc(C(=O)NCc2ccco2)cc(OC)c1OC
4457,COc1nc(-c2ccccc2)ccc1-c1noc(-c2ccco2)n1
4458,CN(c1ccc(OCc2nnc(-c3ccccc3)o2)cc1)S(=O)(=O)c1c...


### load in specific descriptors 

In [27]:
# Function to calculate descriptors
def calculate_descriptors(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return [None] * len(Descriptors.descList)
    return [func(mol) for name, func in Descriptors.descList]

# Calculate descriptors for all SMILES
descriptor_names = [name for name, func in Descriptors.descList]
descriptor_values = data_tested_untested['SMILES'].apply(calculate_descriptors)

# Create a DataFrame with descriptor values
descriptor_df = pd.DataFrame(descriptor_values.tolist(), columns=descriptor_names)

#Uncomment the lines below if you want to run with specific descriptors
#file_path = r"C:\Users\20201527\OneDrive - TU Eindhoven\Desktop\8CC00\Group assignement\specific_descriptors.csv"
#specific_descriptor_df = pd.read_csv(file_path)
#specific_descriptor_df = specific_descriptor_df.drop(columns=['PKM2_inhibition', 'ERK2_inhibition','SMILES'])
#descriptor_df = specific_descriptor_df

descriptor_df

,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,NumRadicalElectrons,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,5.313889,0.120833,5.313889,0.120833,0.592228,151.173,142.101,151.085795,58,0,...,0,0,0,0,0,0,0,0,0,0
1,11.238954,-0.366756,11.238954,0.225308,0.785414,250.681,239.593,250.039672,88,0,...,0,0,0,0,0,0,0,0,0,0
2,11.090706,-0.049610,11.090706,0.049610,0.581062,210.277,192.133,210.136828,84,0,...,0,0,0,0,0,0,0,0,0,0
3,11.892238,-0.457824,11.892238,0.076632,0.441090,232.239,220.143,232.084792,88,0,...,0,0,0,0,0,0,0,0,0,0
4,11.693580,-0.498260,11.693580,0.012315,0.720343,226.663,215.575,226.050905,80,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5571,4.139872,0.673565,4.139872,0.673565,0.653748,163.244,148.124,163.122975,64,0,...,0,0,0,0,0,0,0,0,0,0
5572,12.171096,-0.258655,12.171096,0.258655,0.883501,291.303,274.167,291.110673,112,0,...,0,0,0,0,0,0,0,0,0,0
5573,5.405653,0.309434,5.405653,0.309434,0.564327,319.320,306.216,319.095691,118,0,...,0,0,0,0,0,0,0,0,0,0
5574,12.714600,-3.624520,12.714600,0.108466,0.445804,421.478,402.326,421.109627,152,0,...,0,1,0,0,0,0,0,0,0,0


### Perform PCA and find the thresholds

In [28]:
def check_all_thresholds(constant_bits, threshold_finger_prints, threshold_descriptors):
    np.random.seed(42)
    def smiles_to_fingerprint(smiles, nBits=constant_bits):
        mol = Chem.MolFromSmiles(smiles)
        if mol:
            fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius=2, nBits=nBits)
            return np.array(fp)
        else:
            return np.zeros((nBits,))

    # Apply the function to your dataset with nBits=1024
    fingerprints = data_tested_untested['SMILES'].apply(lambda x: smiles_to_fingerprint(x, nBits=constant_bits))
    fingerprints_matrix = np.stack(fingerprints.values)
    
    # Standardize the fingerprints
    scaler = StandardScaler()
    scaled_fingerprints = scaler.fit_transform(fingerprints_matrix)
    
    # Perform PCA
    pca = PCA()
    pca.fit(scaled_fingerprints)
    
    # Determine the number of components to retain 90% variance
    cumulative_variance = pca.explained_variance_ratio_.cumsum()
    num_components = (cumulative_variance < threshold_finger_prints).sum() + 1
    
    # Transform the dataset using these components
    pca = PCA(n_components=num_components)
    principal_components = pca.fit_transform(scaled_fingerprints)
    
    # Create a DataFrame with the principal components
    pca_df = pd.DataFrame(data=principal_components)

    # Combine PCA and descriptors DataFrames
    combined_df = pd.concat([descriptor_df, pca_df], axis=1)
    combined_df.columns = combined_df.columns.astype(str)

        # Standardize the combined data
    scaler = StandardScaler()
    scaled_combined_data = scaler.fit_transform(combined_df)
    
    # Perform PCA
    pca = PCA()
    pca.fit(scaled_combined_data)
    
    # Determine the number of components to retain 90% variance
    cumulative_variance = pca.explained_variance_ratio_.cumsum()
    num_components = (cumulative_variance < threshold_descriptors).sum() + 1
    
    # Transform the dataset using these components
    pca = PCA(n_components=num_components)
    principal_components = pca.fit_transform(scaled_combined_data)

    final_pca_df = pd.DataFrame(data=principal_components)
    final_pca_df.columns = final_pca_df.columns.astype(str)
    
    return final_pca_df

### Function to train & evaluate the model 

In [29]:
def check_with_model(descriptor_df, model):
    np.random.seed(42)
    # Assuming your descriptor_df is already prepared
    X = descriptor_df.drop(columns=['PKM2_inhibition', 'ERK2_inhibition'])
    y1 = descriptor_df['PKM2_inhibition']
    y2 = descriptor_df['ERK2_inhibition']
    y = pd.concat([y1, y2], axis=1)
    
    # Split the data into training and validation sets
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.5, random_state=42)
    
    # Standardizing the features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)

    # Ignore all warnings
    warnings.filterwarnings("ignore")
    
    # Apply RandomOverSampler to handle imbalance for each target separately
    ros_pkm2 = RandomOverSampler(random_state=42)
    X_train_ros_pkm2, y_train_ros_pkm2 = ros_pkm2.fit_resample(X_train, y_train['PKM2_inhibition'])
    
    ros_erk2 = RandomOverSampler(random_state=42)
    X_train_ros_erk2, y_train_ros_erk2 = ros_erk2.fit_resample(X_train, y_train['ERK2_inhibition'])
    
    # Calculate scale_pos_weight for each target
    scale_pos_weight_pkm2 = y_train_ros_pkm2.value_counts()[0] / y_train_ros_pkm2.value_counts()[1]
    scale_pos_weight_erk2 = y_train_ros_erk2.value_counts()[0] / y_train_ros_erk2.value_counts()[1]
    
    # Create and train the XGBClassifier for PKM2_inhibition
    model_pkm2 = XGBClassifier(scale_pos_weight=scale_pos_weight_pkm2, random_state=42)
    model_pkm2.fit(X_train_ros_pkm2, y_train_ros_pkm2)
    
    # Create and train the XGBClassifier for ERK2_inhibition
    model_erk2 = XGBClassifier(scale_pos_weight=scale_pos_weight_erk2, random_state=42)
    model_erk2.fit(X_train_ros_erk2, y_train_ros_erk2)
    
    # Predict on the validation set
    y_pred_pkm2 = model_pkm2.predict(X_val)
    y_pred_erk2 = model_erk2.predict(X_val)
    
    # Convert y_val to numpy array for comparison
    y_val_binary = y_val.to_numpy()
    
    # Calculate the accuracy, precision, recall, F1 score, and balanced accuracy for each target column
    accuracy_pkm2 = accuracy_score(y_val_binary[:, 0], y_pred_pkm2)
    accuracy_erk2 = accuracy_score(y_val_binary[:, 1], y_pred_erk2)
    
    precision_pkm2 = precision_score(y_val_binary[:, 0], y_pred_pkm2)
    precision_erk2 = precision_score(y_val_binary[:, 1], y_pred_erk2)
    
    recall_pkm2 = recall_score(y_val_binary[:, 0], y_pred_pkm2)
    recall_erk2 = recall_score(y_val_binary[:, 1], y_pred_erk2)
    
    f1_pkm2 = f1_score(y_val_binary[:, 0], y_pred_pkm2)
    f1_erk2 = f1_score(y_val_binary[:, 1], y_pred_erk2)
    
    # Create a DataFrame to store the results
    results = pd.DataFrame({
        'Metric': ['Accuracy', 'Precision', 'Recall', 'F1 Score'],
        'PKM2_inhibition': [accuracy_pkm2, precision_pkm2, recall_pkm2, f1_pkm2],
        'ERK2_inhibition': [accuracy_erk2, precision_erk2, recall_erk2, f1_erk2]
    })
    
    # Transpose the DataFrame for the desired format
    results = results.set_index('Metric').T

    if model == 'pk2':
        joblib.dump(model_pkm2, 'model_pk2.joblib')
        print("Save pk2")
    elif model == 'ek2':
        joblib.dump(model_erk2, 'model_ek2.joblib')
        print("Save ek2")
        
    return results

### Obtain final PCA
##### perform it with all data --> descriptor_df 
##### perform it with specific descriptorn in the data  --> specific_descriptor_df 


In [30]:
def get_best_two_models(pk2_nbits, pk2_fp, pk2_des, ek2_nbits, ek2_fp, ek2_des):
    final_pca_df_pk2_all = check_all_thresholds(pk2_nbits, pk2_fp, pk2_des)
    final_pca_df_ek2_all = check_all_thresholds(ek2_nbits, ek2_fp, ek2_des)

    final_pca_df_tested_pk2 = final_pca_df_pk2_all.iloc[:1116]
    final_pca_df_untested_pk2 = final_pca_df_pk2_all.iloc[1116:]

    final_pca_df_tested_ek2 = final_pca_df_ek2_all.iloc[:1116]
    final_pca_df_untested_ek2 = final_pca_df_ek2_all.iloc[1116:]
    
    # Save the final PCA dataset
    final_pca_df_tested_pk2['PKM2_inhibition'] = data_all['PKM2_inhibition']
    final_pca_df_tested_pk2['ERK2_inhibition'] = data_all['ERK2_inhibition']

    final_pca_df_tested_ek2['PKM2_inhibition'] = data_all['PKM2_inhibition']
    final_pca_df_tested_ek2['ERK2_inhibition'] = data_all['ERK2_inhibition']
    
    results_pk2 = check_with_model(final_pca_df_tested_pk2, 'pk2')
    results_ek2 = check_with_model(final_pca_df_tested_ek2, 'ek2')

    return results_pk2, results_ek2, final_pca_df_untested_pk2, final_pca_df_untested_ek2

### Train and evaluate the model 

In [31]:
results_pk2, results_ek2, untested_pk2, untested_ek2 = get_best_two_models(1024, 0.3, 0.7, 1024, 0.1, 0.6)
results_pk2

Save pk2
Save ek2


Metric,Accuracy,Precision,Recall,F1 Score
PKM2_inhibition,0.973118,0.25,0.076923,0.117647
ERK2_inhibition,0.948029,0.00,0.000000,0.000000


In [32]:
results_ek2

Metric,Accuracy,Precision,Recall,F1 Score
PKM2_inhibition,0.969534,0.250000,0.153846,0.190476
ERK2_inhibition,0.946237,0.166667,0.038462,0.062500


### Load in the testing data 

### specific descriptors for the Test data 

### PCA on the testing data 

### Evaluate the models performance on the test set 

In [36]:

# Load the saved models
model_pk2 = joblib.load('model_pk2.joblib')
model_ek2 = joblib.load('model_ek2.joblib')

# Use the models for prediction
y_pred_pk2 = model_pk2.predict(untested_pk2)
y_pred_ek2 = model_ek2.predict(untested_ek2)

# Create a DataFrame to store the predictions
predictions = pd.DataFrame({
    'PKM2_predicted_inhibition': y_pred_pk2,
    'ERK2_predicted_inhibition': y_pred_ek2
})


# Concatenate 'SMILES' with predictions
predictions2 = pd.concat([untested_data['SMILES'], predictions], axis=1)

# Calculate the counts of each combination
# Assuming you want to count occurrences based on predicted values
counts = predictions2.groupby(['PKM2_predicted_inhibition', 'ERK2_predicted_inhibition']).size().reset_index(name='Count')

# Rename the columns for better understanding
counts.columns = ['PKM2_predicted_inhibition', 'ERK2_predicted_inhibition', 'Count']

# Display the counts
print(counts)

   PKM2_predicted_inhibition  ERK2_predicted_inhibition  Count
0                          0                          0   4375
1                          0                          1     37
2                          1                          0     47
3                          1                          1      1


In [37]:
# Export DataFrame to CSV
try:
    predictions2.to_csv('Predictions_on_untested_data.csv', index=False)  # Set index=False to exclude row numbers from the output
    print("CSV file exported successfully.")
except Exception as e:
    print("Error:", e)

CSV file exported successfully.
